<div align="center">
    <a href="https://colab.research.google.com/github/Kartavya-Jharwal/Kartavya_Business_Analytics2025/blob/main/A1/assignment.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="max-width: 150px; height: auto;">
    </a>
</div>

<hr style="border: 2px solid #3366ff; border-radius: 3px; margin: 20px 0;">

<div style="background: linear-gradient(to right, #e0f7fa, #b2ebf2); padding: 20px; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h1 style="color: #006064; text-align: center; border-bottom: 2px solid #00838f; padding-bottom: 10px;">A1 - Hypothesis Test</h1>
    <h2 style="color: #0097a7; text-align: center; font-style: italic;">Exploring the Relationship Between Economic Indicators and Global Development Outcomes</h2>

    <div style="background: rgba(255,255,255,0.7); padding: 15px; border-radius: 8px; margin-top: 15px;">
        <table style="width: 100%; border-collapse: collapse;">
            <tr>
                <td style="padding: 8px; font-weight: bold; color: #00695c; width: 120px;">Course:</td>
                <td style="padding: 8px;">Fundamentals of Business Analytics - BAN-0200</td>
            </tr>
            <tr>
                <td style="padding: 8px; font-weight: bold; color: #00695c;">Professor:</td>
                <td style="padding: 8px;">Prof Glen Joseph</td>
            </tr>
            <tr>
                <td style="padding: 8px; font-weight: bold; color: #00695c;">Prepared by:</td>
                <td style="padding: 8px;">Kartavya Jharwal</td>
            </tr>
            <tr>
                <td style="padding: 8px; font-weight: bold; color: #00695c;">Due Date:</td>
                <td style="padding: 8px;">October 24, 2025</td>
            </tr>
        </table>
    </div>
</div>

<h2 style="color: #00695c; margin-top: 25px; border-left: 5px solid #009688; padding-left: 10px;">Assignment Overview</h2>

<p style="margin-left: 15px; text-align: justify;">
This assignment explores the relationship between economic prosperity and environmental/social outcomes by examining:
</p>

<ul style="list-style-type: none; margin-left: 30px;">
    <li style="margin-bottom: 8px; position: relative;">
        <span style="display: inline-block; width: 24px; height: 24px; background-color: #4db6ac; color: white; text-align: center; border-radius: 50%; margin-right: 10px; line-height: 24px;">1</span>
        GDP per capita
    </li>
    <li style="margin-bottom: 8px; position: relative;">
        <span style="display: inline-block; width: 24px; height: 24px; background-color: #4db6ac; color: white; text-align: center; border-radius: 50%; margin-right: 10px; line-height: 24px;">2</span>
        CO₂ emissions per capita
    </li>
    <li style="margin-bottom: 8px; position: relative;">
        <span style="display: inline-block; width: 24px; height: 24px; background-color: #4db6ac; color: white; text-align: center; border-radius: 50%; margin-right: 10px; line-height: 24px;">3</span>
        Net-zero carbon emissions targets
    </li>
</ul>

<div style="background-color: #e0f2f1; border-left: 4px solid #00897b; padding: 15px; margin: 20px 0; border-radius: 0 5px 5px 0;">
    <h3 style="margin-top: 0; color: #00695c;">Core Hypothesis (Part 1):</h3>
    <p style="font-weight: bold; font-style: italic; margin-left: 15px;">
        "Countries with higher GDP per capita emit more CO₂ per capita."
    </p>
</div>

<h3 style="color: #00695c; border-bottom: 2px dotted #80cbc4; padding-bottom: 5px;">Objectives:</h3>
<ol style="list-style-position: outside; margin-left: 20px;">
    <li style="margin-bottom: 10px;">
        <span style="font-weight: bold; color: #00796b;">Part 1:</span> Test the core hypothesis using provided GDP and CO₂ datasets
    </li>
    <li style="margin-bottom: 10px;">
        <span style="font-weight: bold; color: #00796b;">Part 2:</span> Extend analysis with net-zero carbon emissions targets and new hypothesis
    </li>
    <li style="margin-bottom: 10px;">
        Apply rigorous statistical methods including confidence intervals and descriptive analytics
    </li>
    <li style="margin-bottom: 10px;">
        Create compelling visualizations to support findings
    </li>
    <li style="margin-bottom: 10px;">
        Provide critical interpretation of results with contextual understanding
    </li>
</ol>

<hr style="border: 1px solid #b2dfdb; margin-top: 20px;">

In [1]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from typing import Tuple, Optional, Dict, List, Union
import warnings
import sys
import platform
from datetime import datetime

warnings.filterwarnings('ignore')

# Set plotting style and parameters
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Environment and system information
print("ASSIGNMENT A1 - BUSINESS ANALYTICS")
print("="*60)
print("Execution Date: " + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print("Python Version: " + sys.version)
print("Platform: " + platform.platform())
print("Architecture: " + platform.architecture()[0])

print("\n" + "="*60)
print("LIBRARY VERSIONS")
print("="*60)
print("✓ Pandas: " + pd.__version__)
print("✓ NumPy: " + np.__version__)
print("✓ Matplotlib: " + plt.matplotlib.__version__)
print("✓ Seaborn: " + sns.__version__)
print("✓ SciPy: " + (stats.__version__ if hasattr(stats, '__version__') else 'Available'))

# Check if running in Google Colab
try:
    import google.colab
    print("✓ Google Colab: Detected")
    colab_env = True
except ImportError:
    print("✓ Environment: Local/Other")
    colab_env = False

print("="*60)
print("LIBRARIES IMPORTED SUCCESSFULLY!")
print("="*60)

# GitHub repository base URL for data loading (Colab compatibility)
github_base = "https://raw.githubusercontent.com/Kartavya-Jharwal/Kartavya_Business_Analytics2025/refs/heads/main/A1"
print("GitHub base URL configured: " + github_base)

ASSIGNMENT A1 - BUSINESS ANALYTICS
Execution Date: 2025-09-22 15:55:35
Python Version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35
Architecture: 64bit

LIBRARY VERSIONS
✓ Pandas: 2.2.2
✓ NumPy: 2.0.2
✓ Matplotlib: 3.10.0
✓ Seaborn: 0.13.2
✓ SciPy: Available
✓ Google Colab: Detected
LIBRARIES IMPORTED SUCCESSFULLY!
GitHub base URL configured: https://raw.githubusercontent.com/Kartavya-Jharwal/Kartavya_Business_Analytics2025/refs/heads/main/A1


<div style="background: linear-gradient(to right, #fff3e0, #ffe0b2); padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
    <h1 style="color: #e65100; text-align: center; text-transform: uppercase; letter-spacing: 2px; border-bottom: 3px solid #ff9800; padding-bottom: 10px;">
        PART 1: Hypothesis Testing with Provided Datasets
    </h1>
    
    <div style="background: rgba(255,255,255,0.7); padding: 15px; border-radius: 8px; margin: 15px 0;">
        <h2 style="color: #e65100; margin-top: 0;">Core Hypothesis:</h2>
        <blockquote style="border-left: 4px solid #ff9800; padding-left: 15px; margin-left: 10px; font-size: 18px; font-weight: bold; font-style: italic;">
            "Countries with higher GDP per capita emit more CO₂ per capita."
        </blockquote>
    </div>
</div>

<h3 style="color: #e65100; margin-top: 25px; border-left: 5px solid #ff9800; padding-left: 10px;">Datasets to be analyzed:</h3>

<div style="display: flex; flex-wrap: wrap; gap: 20px; margin: 20px 0;">
    <div style="flex: 1; min-width: 300px; background-color: #fff8e1; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #ffb74d;">
        <h4 style="margin-top: 0; color: #e65100;">1. CO₂ Emissions per Capita</h4>
        <div style="background-color: #fafafa; border-radius: 5px; padding: 8px; font-family: monospace; font-size: 14px;">co-emissions-per-capita/co-emissions-per-capita.csv</div>
        <p style="font-size: 14px; margin-top: 10px; color: #555;">
            <strong>Source:</strong> Global Carbon Budget (2024), Population based on various sources (2024) – with major processing by Our World in Data
        </p>
    </div>
    
    <div style="flex: 1; min-width: 300px; background-color: #fff8e1; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #ffb74d;">
        <h4 style="margin-top: 0; color: #e65100;">2. GDP per Capita in Constant USD</h4>
        <div style="background-color: #fafafa; border-radius: 5px; padding: 8px; font-family: monospace; font-size: 14px;">gdp-per-capita-worldbank-constant-usd/gdp-per-capita-worldbank-constant-usd.csv</div>
        <p style="font-size: 14px; margin-top: 10px; color: #555;">
            <strong>Source:</strong> National statistical organizations and central banks, OECD national accounts, and World Bank staff estimates (2025) – with minor processing by Our World in Data
        </p>
    </div>
</div>

<h3 style="color: #e65100; margin-top: 25px; border-bottom: 2px dotted #ffb74d; padding-bottom: 5px;">Analysis Steps:</h3>

<div style="background-color: #fff3e0; border-radius: 8px; padding: 15px; margin: 15px 0; box-shadow: inset 0 0 5px rgba(0,0,0,0.05);">
    <ol style="counter-reset: item; margin-left: 10px; padding-left: 0;">
        <li style="display: flex; align-items: center; margin-bottom: 12px; counter-increment: item;">
            <div style="width: 30px; height: 30px; background-color: #ff9800; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 10px; font-weight: bold;">
                1
            </div>
            <span>Load and inspect both datasets</span>
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 12px; counter-increment: item;">
            <div style="width: 30px; height: 30px; background-color: #ff9800; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 10px; font-weight: bold;">
                2
            </div>
            <span>Clean and standardize the data</span>
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 12px; counter-increment: item;">
            <div style="width: 30px; height: 30px; background-color: #ff9800; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 10px; font-weight: bold;">
                3
            </div>
            <span>Merge datasets on Country and Year</span>
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 12px; counter-increment: item;">
            <div style="width: 30px; height: 30px; background-color: #ff9800; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 10px; font-weight: bold;">
                4
            </div>
            <span>Create GDP categories (Low, Medium, High)</span>
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 12px; counter-increment: item;">
            <div style="width: 30px; height: 30px; background-color: #ff9800; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 10px; font-weight: bold;">
                5
            </div>
            <span>Calculate descriptive statistics with confidence intervals</span>
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 12px; counter-increment: item;">
            <div style="width: 30px; height: 30px; background-color: #ff9800; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 10px; font-weight: bold;">
                6
            </div>
            <span>Create visualizations</span>
        </li>
        <li style="display: flex; align-items: center; margin-bottom: 12px; counter-increment: item;">
            <div style="width: 30px; height: 30px; background-color: #ff9800; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 10px; font-weight: bold;">
                7
            </div>
            <span>Interpret results</span>
        </li>
    </ol>
</div>

<hr style="border: 2px solid #ffcc80; border-radius: 2px; margin: 25px 0;">

<h2 style="color: #e65100; text-align: center; padding: 10px; background-color: #fff8e1; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
    Step 1: Load and Inspect Datasets
</h2>

In [2]:
# Load the datasets
# Optimized for Google Colab - loading directly from GitHub

# GitHub raw URLs for datasets
co2_github_url = github_base + "/co-emissions-per-capita/co-emissions-per-capita.csv"
gdp_github_url = github_base + "/gdp-per-capita-worldbank-constant-usd/gdp-per-capita-worldbank-constant-usd.csv"

print("Loading datasets from GitHub repository...")

# Load CO2 emissions dataset
try:
    co2_df = pd.read_csv(co2_github_url)
    print("✓ CO2 emissions dataset loaded successfully")
    print("CO2 dataset shape: " + str(co2_df.shape))
except Exception as e:
    print("❌ Failed to load CO2 emissions dataset")
    print("Error: " + str(e))
    co2_df = None

# Load GDP dataset
try:
    gdp_df = pd.read_csv(gdp_github_url)
    print("✓ GDP dataset loaded successfully")
    print("GDP dataset shape: " + str(gdp_df.shape))
except Exception as e:
    print("❌ Failed to load GDP dataset")
    print("Error: " + str(e))
    gdp_df = None

Loading datasets from GitHub repository...
✓ CO2 emissions dataset loaded successfully
CO2 dataset shape: (26317, 4)
✓ GDP dataset loaded successfully
GDP dataset shape: (12098, 4)


In [3]:
# Inspect the structure of both datasets (when available)
if co2_df is not None:
    print("=== CO2 EMISSIONS DATASET ===")
    print("\nFirst 5 rows:")
    display(co2_df.head())

    print(f"\nDataset info:")
    co2_df.info()

    print(f"\nSummary statistics:")
    display(co2_df.describe(include='all'))

    print(f"\nMissing values:")
    print(co2_df.isnull().sum())

    print("Year range: " + str(co2_df['Year'].min() if 'Year' in co2_df.columns else 'Year column not found') + " - " + str(co2_df['Year'].max() if 'Year' in co2_df.columns else ''))

if gdp_df is not None:
    print("\n\n=== GDP DATASET ===")
    print("\nFirst 5 rows:")
    display(gdp_df.head())

    print(f"\nDataset info:")
    gdp_df.info()

    print(f"\nSummary statistics:")
    display(gdp_df.describe(include='all'))

    print(f"\nMissing values:")
    print(gdp_df.isnull().sum())

    print("Year range: " + str(gdp_df['Year'].min() if 'Year' in gdp_df.columns else 'Year column not found') + " - " + str(gdp_df['Year'].max() if 'Year' in gdp_df.columns else ''))

=== CO2 EMISSIONS DATASET ===

First 5 rows:


,Entity,Code,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,AFG,1949,0.001992
1,Afghanistan,AFG,1950,0.010837
2,Afghanistan,AFG,1951,0.011625
3,Afghanistan,AFG,1952,0.011468
4,Afghanistan,AFG,1953,0.013123



Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26317 entries, 0 to 26316
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Entity                             26317 non-null  object 
 1   Code                               23030 non-null  object 
 2   Year                               26317 non-null  int64  
 3   Annual CO₂ emissions (per capita)  26317 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 822.5+ KB

Summary statistics:


,Entity,Code,Year,Annual CO₂ emissions (per capita)
count,26317,23030,26317.000000,26317.000000
unique,231,215,NaN,NaN
top,Asia,AUS,NaN,NaN
freq,229,229,NaN,NaN
mean,NaN,NaN,1952.441768,3.806824
std,NaN,NaN,53.412417,14.550548
min,NaN,NaN,1750.000000,0.000000
25%,NaN,NaN,1919.000000,0.168055
50%,NaN,NaN,1965.000000,1.008440
75%,NaN,NaN,1995.000000,4.277874



Missing values:
Entity                                  0
Code                                 3287
Year                                    0
Annual CO₂ emissions (per capita)       0
dtype: int64
Year range: 1750 - 2023


=== GDP DATASET ===

First 5 rows:


,Entity,Code,Year,GDP per capita (constant 2015 US$)
0,Afghanistan,AFG,2000,308.31827
1,Afghanistan,AFG,2001,277.11804
2,Afghanistan,AFG,2002,338.13998
3,Afghanistan,AFG,2003,346.07162
4,Afghanistan,AFG,2004,338.63727



Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12098 entries, 0 to 12097
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Entity                              12098 non-null  object 
 1   Code                                11338 non-null  object 
 2   Year                                12098 non-null  int64  
 3   GDP per capita (constant 2015 US$)  12098 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 378.2+ KB

Summary statistics:


,Entity,Code,Year,GDP per capita (constant 2015 US$)
count,12098,11338,12098.000000,12098.000000
unique,225,213,NaN,NaN
top,Algeria,DZA,NaN,NaN
freq,65,65,NaN,NaN
mean,NaN,NaN,1995.296165,12354.112602
std,NaN,NaN,17.977459,19040.224744
min,NaN,NaN,1960.000000,122.678900
25%,NaN,NaN,1981.000000,1365.685725
50%,NaN,NaN,1997.000000,4025.749250
75%,NaN,NaN,2011.000000,16017.898500



Missing values:
Entity                                  0
Code                                  760
Year                                    0
GDP per capita (constant 2015 US$)      0
dtype: int64
Year range: 1960 - 2024


<div style="background: linear-gradient(to right, #e8eaf6, #c5cae9); padding: 15px; border-radius: 10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1);">
    <h2 style="color: #3f51b5; text-align: center; margin-bottom: 20px; text-transform: uppercase; letter-spacing: 1px;">
        Step 2: Clean and Standardize Data
    </h2>
    
    <div style="background-color: rgba(255,255,255,0.7); padding: 15px; border-radius: 8px;">
        <p style="font-size: 16px; color: #303f9f; margin-bottom: 15px; font-weight: 500;">
            Before merging the datasets, we need to:
        </p>
        
        <div style="display: flex; flex-direction: column; gap: 12px;">
            <div style="display: flex; align-items: center; background-color: #e8eaf6; padding: 10px; border-radius: 6px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <div style="width: 30px; height: 30px; background-color: #3f51b5; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; flex-shrink: 0;">
                    1
                </div>
                <div>
                    <span style="font-weight: 500;">Standardize country names between datasets</span>
                    <div style="height: 4px; width: 50px; background-color: #7986cb; margin-top: 5px; border-radius: 2px;"></div>
                </div>
            </div>
            
            <div style="display: flex; align-items: center; background-color: #e8eaf6; padding: 10px; border-radius: 6px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <div style="width: 30px; height: 30px; background-color: #3f51b5; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; flex-shrink: 0;">
                    2
                </div>
                <div>
                    <span style="font-weight: 500;">Identify overlapping years</span>
                    <div style="height: 4px; width: 50px; background-color: #7986cb; margin-top: 5px; border-radius: 2px;"></div>
                </div>
            </div>
            
            <div style="display: flex; align-items: center; background-color: #e8eaf6; padding: 10px; border-radius: 6px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <div style="width: 30px; height: 30px; background-color: #3f51b5; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; flex-shrink: 0;">
                    3
                </div>
                <div>
                    <span style="font-weight: 500;">Handle missing or inconsistent data points</span>
                    <div style="height: 4px; width: 50px; background-color: #7986cb; margin-top: 5px; border-radius: 2px;"></div>
                </div>
            </div>
            
            <div style="display: flex; align-items: center; background-color: #e8eaf6; padding: 10px; border-radius: 6px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                <div style="width: 30px; height: 30px; background-color: #3f51b5; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; flex-shrink: 0;">
                    4
                </div>
                <div>
                    <span style="font-weight: 500;">Ensure data quality for meaningful analysis</span>
                    <div style="height: 4px; width: 50px; background-color: #7986cb; margin-top: 5px; border-radius: 2px;"></div>
                </div>
            </div>
        </div>
    </div>
</div>

In [4]:
# Data cleaning and standardization function
def clean_and_standardize_data(co2_df: Optional[pd.DataFrame], gdp_df: Optional[pd.DataFrame]) -> Tuple[Optional[pd.DataFrame], Optional[pd.DataFrame]]:
    """
    Clean and standardize both datasets for analysis

    Args:
        co2_df: DataFrame containing CO2 emissions data
        gdp_df: DataFrame containing GDP data

    Returns:
        Tuple of cleaned CO2 and GDP DataFrames
    """
    if co2_df is None or gdp_df is None:
        print("Cannot proceed with cleaning - datasets not loaded")
        return None, None

    # Make copies to avoid modifying original data
    co2_clean = co2_df.copy()
    gdp_clean = gdp_df.copy()

    print("=== DATA CLEANING REPORT ===")

    # Check column names and standardize if needed
    print("CO2 columns: " + str(list(co2_clean.columns)))
    print("GDP columns: " + str(list(gdp_clean.columns)))

    # Remove rows with missing critical data
    initial_co2_rows = len(co2_clean)
    initial_gdp_rows = len(gdp_clean)

    # Drop rows where key columns are missing
    # Note: Actual column names may vary - this is template code
    if 'Entity' in co2_clean.columns:
        co2_clean = co2_clean.dropna(subset=['Entity', 'Year'])
    if 'Entity' in gdp_clean.columns:
        gdp_clean = gdp_clean.dropna(subset=['Entity', 'Year'])

    print("CO2 data: " + str(initial_co2_rows) + " → " + str(len(co2_clean)) + " rows after cleaning")
    print("GDP data: " + str(initial_gdp_rows) + " → " + str(len(gdp_clean)) + " rows after cleaning")

    # Check for overlapping years
    if 'Year' in co2_clean.columns and 'Year' in gdp_clean.columns:
        co2_years = set(co2_clean['Year'].unique())
        gdp_years = set(gdp_clean['Year'].unique())
        overlap_years = co2_years.intersection(gdp_years)

        print("CO2 year range: " + str(min(co2_years)) + " - " + str(max(co2_years)))
        print("GDP year range: " + str(min(gdp_years)) + " - " + str(max(gdp_years)))
        print("Overlapping years: " + str(len(overlap_years)) + " years from " + str(min(overlap_years)) + " to " + str(max(overlap_years)))

    # Check for common countries
    if 'Entity' in co2_clean.columns and 'Entity' in gdp_clean.columns:
        co2_countries = set(co2_clean['Entity'].unique())
        gdp_countries = set(gdp_clean['Entity'].unique())
        common_countries = co2_countries.intersection(gdp_countries)

        print("Countries in CO2 data: " + str(len(co2_countries)))
        print("Countries in GDP data: " + str(len(gdp_countries)))
        print("Common countries: " + str(len(common_countries)))

        # Show some examples of countries that don't match
        co2_only = co2_countries - gdp_countries
        gdp_only = gdp_countries - co2_countries

        if co2_only:
            print("Examples of countries only in CO2 data: " + str(list(co2_only)[:5]))
        if gdp_only:
            print("Examples of countries only in GDP data: " + str(list(gdp_only)[:5]))

    return co2_clean, gdp_clean

# Execute cleaning
co2_clean, gdp_clean = clean_and_standardize_data(co2_df, gdp_df)

=== DATA CLEANING REPORT ===
CO2 columns: ['Entity', 'Code', 'Year', 'Annual CO₂ emissions (per capita)']
GDP columns: ['Entity', 'Code', 'Year', 'GDP per capita (constant 2015 US$)']
CO2 data: 26317 → 26317 rows after cleaning
GDP data: 12098 → 12098 rows after cleaning
CO2 year range: 1750 - 2023
GDP year range: 1960 - 2024
Overlapping years: 64 years from 1960 to 2023
Countries in CO2 data: 231
Countries in GDP data: 225
Common countries: 208
Examples of countries only in CO2 data: ['Bonaire Sint Eustatius and Saba', 'Anguilla', 'Europe (excl. EU-28)', 'Venezuela', 'Saint Pierre and Miquelon']
Examples of countries only in GDP data: ['South Asia (WB)', 'Sub-Saharan Africa (WB)', 'Cayman Islands', 'Northern Mariana Islands', 'United States Virgin Islands']


<div style="background: linear-gradient(to right, #f3e5f5, #e1bee7); padding: 15px; border-radius: 10px; box-shadow: 0 3px 6px rgba(0,0,0,0.1); margin-top: 20px;">
    <h2 style="color: #7b1fa2; text-align: center; border-bottom: 2px solid #ba68c8; padding-bottom: 10px;">
        <span style="display: inline-block; background-color: #9c27b0; color: white; width: 35px; height: 35px; line-height: 35px; text-align: center; border-radius: 50%; margin-right: 10px;">3</span>
        Step 3: Merge Datasets
    </h2>
    
    <div style="background-color: rgba(255,255,255,0.7); padding: 15px; border-radius: 8px; margin-top: 15px; position: relative;">
        <div style="position: absolute; top: -10px; left: 20px; background-color: #9c27b0; color: white; padding: 2px 10px; border-radius: 20px; font-size: 12px; font-weight: bold;">
            DATA INTEGRATION
        </div>
        
        <p style="margin: 15px 0 0 0; text-align: justify; line-height: 1.6;">
            We'll merge the cleaned CO₂ and GDP datasets on Country and Year to create our analysis dataset. This step is critical for establishing the relationship between economic indicators and emissions.
        </p>
        
        <div style="display: flex; align-items: center; margin-top: 20px; background-color: #f8f0fc; padding: 10px; border-radius: 5px; border-left: 4px solid #9c27b0;">
            <div style="flex-shrink: 0; margin-right: 15px;">
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="#9c27b0">
                    <path d="M17 20.41L18.41 19 15 15.59 13.59 17 17 20.41zM7.5 8H11v5.59L5.59 19 7 20.41l6-6V8h3.5L12 3.5 7.5 8z"/>
                </svg>
            </div>
            <div>
                <strong style="color: #6a1b9a;">Key Operations:</strong>
                <ul style="margin-top: 5px; margin-bottom: 0; padding-left: 20px;">
                    <li>Join on matching 'Entity' (country) and 'Year' columns</li>
                    <li>Handle potential many-to-many relationships</li>
                    <li>Create a unified analysis-ready dataset</li>
                </ul>
            </div>
        </div>
    </div>
</div>

In [5]:
# Merge the datasets
def merge_datasets(co2_clean: Optional[pd.DataFrame], gdp_clean: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    """
    Merge CO2 and GDP datasets on Country and Year

    Args:
        co2_clean: Cleaned CO2 emissions DataFrame
        gdp_clean: Cleaned GDP DataFrame

    Returns:
        Merged DataFrame or None if merge fails
    """
    if co2_clean is None or gdp_clean is None:
        print("Cannot merge - cleaned datasets not available")
        return None

    print("=== MERGING DATASETS ===")

    # Determine the correct column names for merging
    # This will need to be adjusted based on actual column names
    country_col_co2 = 'Entity' if 'Entity' in co2_clean.columns else 'Country'
    country_col_gdp = 'Entity' if 'Entity' in gdp_clean.columns else 'Country'

    # Rename columns for consistency if needed
    co2_merge = co2_clean.copy()
    gdp_merge = gdp_clean.copy()

    if country_col_co2 != 'Country':
        co2_merge = co2_merge.rename(columns={country_col_co2: 'Country'})
    if country_col_gdp != 'Country':
        gdp_merge = gdp_merge.rename(columns={country_col_gdp: 'Country'})

    # Perform inner join to keep only matching records
    merged_df = pd.merge(
        co2_merge,
        gdp_merge,
        on=['Country', 'Year'],
        how='inner',
        suffixes=('_co2', '_gdp')
    )

    print("CO2 dataset rows: " + str(len(co2_merge)))
    print("GDP dataset rows: " + str(len(gdp_merge)))
    print("Merged dataset rows: " + str(len(merged_df)))
    print("Merged dataset columns: " + str(list(merged_df.columns)))

    # Check for successful merge
    if len(merged_df) > 0:
        print("✓ Successfully merged datasets")
        print("Year range in merged data: " + str(merged_df['Year'].min()) + " - " + str(merged_df['Year'].max()))
        print("Number of unique countries: " + str(merged_df['Country'].nunique()))
        print("Countries: " + str(sorted(merged_df['Country'].unique())[:10]) + "...") # Show first 10
    else:
        print("❌ Merge resulted in empty dataset - check column names and data compatibility")

    return merged_df

# Execute merge
merged_data = merge_datasets(co2_clean, gdp_clean)

=== MERGING DATASETS ===
CO2 dataset rows: 26317
GDP dataset rows: 12098
Merged dataset rows: 11001
Merged dataset columns: ['Country', 'Code_co2', 'Year', 'Annual CO₂ emissions (per capita)', 'Code_gdp', 'GDP per capita (constant 2015 US$)']
✓ Successfully merged datasets
Year range in merged data: 1960 - 2023
Number of unique countries: 208
Countries: ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia']...


## Step 4: Feature Engineering - GDP Categories

Create GDP categories to analyze the relationship between economic prosperity levels and CO₂ emissions:
- **Low GDP:** < $5,000 per capita
- **Medium GDP:** $5,000 - $15,000 per capita  
- **High GDP:** > $15,000 per capita

In [ ]:
# Feature Engineering: Create GDP categories
def create_gdp_categories(merged_data: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    """
    Create GDP_Label column with Low, Medium, High categories

    Args:
        merged_data: Merged DataFrame containing GDP and CO2 data

    Returns:
        DataFrame with GDP categories added or None if processing fails
    """
    if merged_data is None or len(merged_data) == 0:
        print("Cannot create GDP categories - merged data not available")
        return None

    # Make a copy to avoid modifying original data
    analysis_df = merged_data.copy()

    print("=== FEATURE ENGINEERING: GDP CATEGORIES ===")

    # Find the GDP column (name may vary based on dataset)
    gdp_columns: List[str] = [col for col in analysis_df.columns if 'gdp' in col.lower() and 'capita' in col.lower()]
    print("Potential GDP columns: " + str(gdp_columns))
    
    # Select the correct GDP column - must contain both 'gdp' and 'capita' but not 'label'
    gdp_col: Optional[str] = None
    for col in analysis_df.columns:
        if 'gdp' in col.lower() and 'capita' in col.lower() and 'label' not in col.lower():
            gdp_col = col
            break
    
    if gdp_col is None:
        print("❌ Could not identify GDP column. Available columns:")
        print(list(analysis_df.columns))
        return analysis_df
    
    print("Using GDP column: '" + gdp_col + "'")
    # Remove any non-numeric values and handle missing data
    analysis_df[gdp_col] = pd.to_numeric(analysis_df[gdp_col], errors='coerce')

    # Create GDP categories
    def categorize_gdp(gdp_value: float) -> str:
        """
        Categorize GDP values into Low, Medium, High categories

        Args:
            gdp_value: GDP per capita value

        Returns:
            String category ('Low', 'Medium', 'High', or 'Unknown')
        """
        if pd.isna(gdp_value):
            return 'Unknown'
        elif gdp_value < 5000:
            return 'Low'
        elif gdp_value <= 15000:
            return 'Medium'
        else:
            return 'High'

    analysis_df['GDP_Label'] = analysis_df[gdp_col].apply(categorize_gdp)

    # Report on categorization
    category_counts = analysis_df['GDP_Label'].value_counts()
    print("\n\nGDP Category Distribution:")
    for category, count in category_counts.items():
        percentage = (count / len(analysis_df)) * 100
        print("  " + category + ": " + str(count) + " (" + str(round(percentage, 1)) + "%)")

    # Show some statistics
    print("\n\nGDP Statistics by Category:")
    gdp_stats = analysis_df.groupby('GDP_Label')[gdp_col].agg(['count', 'mean', 'median', 'std']).round(2)
    print(gdp_stats)

    # Remove rows with unknown GDP (if any)
    analysis_df = analysis_df[analysis_df['GDP_Label'] != 'Unknown']
    print("\n\nFinal dataset shape after removing unknowns: " + str(analysis_df.shape))

    return analysis_df

# Execute feature engineering
analysis_data = create_gdp_categories(merged_data)

=== FEATURE ENGINEERING: GDP CATEGORIES ===
Potential GDP columns: ['Annual CO₂ emissions (per capita)', 'Code_gdp', 'GDP per capita (constant 2015 US$)']
Using GDP column: 'Annual CO₂ emissions (per capita)'
\nGDP Category Distribution:
  Low: 11001 (100.0%)
\nGDP Statistics by Category:
           count  mean  median   std
GDP_Label                           
Low        11001  4.61    1.95  7.68
\nFinal dataset shape after removing unknowns: (11001, 7)


## Step 5: Comprehensive Statistical Analysis

Calculate both descriptive and inferential statistics for CO₂ emissions by GDP category and year, including:

**Descriptive Statistics:**
- Mean, median, standard deviation, variance
- Minimum, maximum, coefficient of variation
- Standard error of the mean (SEM)
- 95% confidence intervals

**Inferential Statistics:**
- Normality tests (Shapiro-Wilk)
- One-way ANOVA for group differences
- Pairwise t-tests (Welch's method)
- Effect sizes (Cohen's d)
- Correlation analysis (Pearson and Spearman)

In [ ]:
# Descriptive Analytics: Calculate statistics with confidence intervals
def calculate_descriptive_stats(analysis_data: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    """
    Calculate descriptive and inferential statistics for CO2 emissions by GDP band and year

    Args:
        analysis_data: DataFrame with GDP categories and CO2 emissions data

    Returns:
        DataFrame with grouped statistics including confidence intervals
    """
    if analysis_data is None or len(analysis_data) == 0:
        print("Cannot calculate statistics - analysis data not available")
        return None

    print("=== DESCRIPTIVE ANALYTICS ===")

    # Find the CO2 column - must contain 'co2' or 'emission' but not 'code'
    co2_columns = [col for col in analysis_data.columns if ('co2' in col.lower() or 'emission' in col.lower()) and 'code' not in col.lower()]
    print("Potential CO2 columns: " + str(co2_columns))

    co2_col = None
    for col in analysis_data.columns:
        if ('co2' in col.lower() or 'emission' in col.lower()) and 'code' not in col.lower():
            co2_col = col
            break

    if co2_col is None:
        print("❌ Could not identify CO2 column. Available columns:")
        print(list(analysis_data.columns))
        return None

    print("Using CO2 column: '" + co2_col + "'")

    # Convert to numeric and remove missing values
    analysis_data[co2_col] = pd.to_numeric(analysis_data[co2_col], errors='coerce')
    clean_data = analysis_data.dropna(subset=[co2_col])

    # Group by GDP_Label and Year, calculate statistics
    grouped_stats: pd.DataFrame = clean_data.groupby(['GDP_Label', 'Year'])[co2_col].agg([
        'count',    # sample size
        'mean',     # mean
        'median',   # median
        'std',      # standard deviation
        'min',      # minimum
        'max',      # maximum
    ]).reset_index()

    # Calculate additional descriptive measures
    grouped_stats['variance'] = grouped_stats['std'] ** 2
    grouped_stats['cv'] = (grouped_stats['std'] / grouped_stats['mean']) * 100  # Coefficient of variation

    # Calculate Standard Error of the Mean (SEM)
    grouped_stats['sem'] = grouped_stats['std'] / np.sqrt(grouped_stats['count'])

    # Calculate 95% confidence intervals: mean ± 1.96 × SEM
    grouped_stats['ci_lower'] = grouped_stats['mean'] - 1.96 * grouped_stats['sem']
    grouped_stats['ci_upper'] = grouped_stats['mean'] + 1.96 * grouped_stats['sem']

    # Round for better display
    numeric_cols: List[str] = ['mean', 'median', 'std', 'variance', 'cv', 'sem', 'ci_lower', 'ci_upper']
    grouped_stats[numeric_cols] = grouped_stats[numeric_cols].round(3)

    print("\n\nCalculated statistics for " + str(len(grouped_stats)) + " GDP_Label-Year combinations")
    print("Sample data (first 10 rows):")
    print(grouped_stats.head(10))

    # Summary by GDP category (across all years) - DESCRIPTIVE STATISTICS
    print("\n\n=== SUMMARY BY GDP CATEGORY (All Years) ===")
    overall_stats: pd.DataFrame = clean_data.groupby('GDP_Label')[co2_col].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(3)
    overall_stats['variance'] = (overall_stats['std'] ** 2).round(3)
    overall_stats['cv'] = ((overall_stats['std'] / overall_stats['mean']) * 100).round(3)
    overall_stats['sem'] = (overall_stats['std'] / np.sqrt(overall_stats['count'])).round(3)
    overall_stats['ci_lower'] = (overall_stats['mean'] - 1.96 * overall_stats['sem']).round(3)
    overall_stats['ci_upper'] = (overall_stats['mean'] + 1.96 * overall_stats['sem']).round(3)

    print(overall_stats)

    # INFERENTIAL STATISTICS
    print("\n\n=== INFERENTIAL STATISTICS ===")

    # Extract data for each GDP category
    low_gdp_data: np.ndarray = clean_data[clean_data['GDP_Label'] == 'Low'][co2_col].values
    medium_gdp_data: np.ndarray = clean_data[clean_data['GDP_Label'] == 'Medium'][co2_col].values
    high_gdp_data: np.ndarray = clean_data[clean_data['GDP_Label'] == 'High'][co2_col].values

    # Perform statistical tests only if we have data for all groups
    if len(low_gdp_data) > 0 and len(medium_gdp_data) > 0 and len(high_gdp_data) > 0:

        # 1. Normality tests (Shapiro-Wilk)
        print("\n\n1. NORMALITY TESTS (Shapiro-Wilk):")
        shapiro_low = stats.shapiro(low_gdp_data[:min(5000, len(low_gdp_data))])
        shapiro_medium = stats.shapiro(medium_gdp_data[:min(5000, len(medium_gdp_data))])
        shapiro_high = stats.shapiro(high_gdp_data[:min(5000, len(high_gdp_data))])

        print("Low GDP: W=" + str(round(shapiro_low.statistic, 4)) + ", p=" + str(round(shapiro_low.pvalue, 6)))
        print("Medium GDP: W=" + str(round(shapiro_medium.statistic, 4)) + ", p=" + str(round(shapiro_medium.pvalue, 6)))
        print("High GDP: W=" + str(round(shapiro_high.statistic, 4)) + ", p=" + str(round(shapiro_high.pvalue, 6)))

        # 2. ANOVA test for differences between groups
        print("\n\n2. ONE-WAY ANOVA TEST:")
        try:
            f_stat, p_value = stats.f_oneway(low_gdp_data, medium_gdp_data, high_gdp_data)
            print("F-statistic: " + str(round(f_stat, 4)))
            print("p-value: " + str(round(p_value, 6)))
            if p_value < 0.05:
                print("Result: Significant differences between GDP groups (p < 0.05)")
            else:
                print("Result: No significant differences between GDP groups (p >= 0.05)")
        except Exception as e:
            print("ANOVA test failed: " + str(e))

        # 3. Pairwise t-tests (Welch's t-test for unequal variances)
        print("\n\n3. PAIRWISE T-TESTS (Welch's t-test):")

        # Low vs Medium
        t_stat_lm, p_val_lm = stats.ttest_ind(low_gdp_data, medium_gdp_data, equal_var=False)
        print("Low vs Medium GDP: t=" + str(round(t_stat_lm, 4)) + ", p=" + str(round(p_val_lm, 6)))

        # Low vs High
        t_stat_lh, p_val_lh = stats.ttest_ind(low_gdp_data, high_gdp_data, equal_var=False)
        print("Low vs High GDP: t=" + str(round(t_stat_lh, 4)) + ", p=" + str(round(p_val_lh, 6)))

        # Medium vs High
        t_stat_mh, p_val_mh = stats.ttest_ind(medium_gdp_data, high_gdp_data, equal_var=False)
        print("Medium vs High GDP: t=" + str(round(t_stat_mh, 4)) + ", p=" + str(round(p_val_mh, 6)))

        # 4. Effect size (Cohen's d) calculations
        print("\n\n4. EFFECT SIZES (Cohen's d):")

        def cohens_d(group1: np.ndarray, group2: np.ndarray) -> float:
            """
            Calculate Cohen's d effect size

            Args:
                group1: First group data
                group2: Second group data

            Returns:
                Cohen's d effect size
            """
            n1, n2 = len(group1), len(group2)
            pooled_std = np.sqrt(((n1 - 1) * np.std(group1, ddof=1) ** 2 +
                                 (n2 - 1) * np.std(group2, ddof=1) ** 2) / (n1 + n2 - 2))
            return (np.mean(group1) - np.mean(group2)) / pooled_std

        d_low_med = cohens_d(medium_gdp_data, low_gdp_data)
        d_low_high = cohens_d(high_gdp_data, low_gdp_data)
        d_med_high = cohens_d(high_gdp_data, medium_gdp_data)

        print("Low vs Medium GDP Cohen's d: " + str(round(d_low_med, 4)))
        print("Low vs High GDP Cohen's d: " + str(round(d_low_high, 4)))
        print("Medium vs High GDP Cohen's d: " + str(round(d_med_high, 4)))

        print("\n\nEffect size interpretation:")
        print("Small effect: |d| = 0.2, Medium effect: |d| = 0.5, Large effect: |d| = 0.8")

        # 5. Correlation analysis
        print("\n\n5. CORRELATION ANALYSIS:")

        # Find GDP column for correlation
        gdp_col_corr = None
        for col in clean_data.columns:
            if any(keyword in col.lower() for keyword in ['gdp', 'capita']) and 'label' not in col.lower():
                gdp_col_corr = col
                break

        if gdp_col_corr is not None:
            # Pearson correlation
            corr_pearson, p_pearson = stats.pearsonr(clean_data[gdp_col_corr], clean_data[co2_col])
            print("Pearson correlation (GDP vs CO2): r=" + str(round(corr_pearson, 4)) + ", p=" + str(round(p_pearson, 6)))

            # Spearman correlation (non-parametric)
            corr_spearman, p_spearman = stats.spearmanr(clean_data[gdp_col_corr], clean_data[co2_col])
            print("Spearman correlation (GDP vs CO2): ρ=" + str(round(corr_spearman, 4)) + ", p=" + str(round(p_spearman, 6)))

    else:
        print("\n\nInsufficient data in one or more GDP categories for inferential statistics")
        print("Low GDP samples: " + str(len(low_gdp_data)))
        print("Medium GDP samples: " + str(len(medium_gdp_data)))
        print("High GDP samples: " + str(len(high_gdp_data)))

    return grouped_stats

# Execute descriptive analytics
descriptive_stats = calculate_descriptive_stats(analysis_data)

=== DESCRIPTIVE ANALYTICS ===
Potential CO2 columns: ['Code_co2', 'Annual CO₂ emissions (per capita)']
Using CO2 column: 'Code_co2'
\nCalculated statistics for 0 GDP_Label-Year combinations
Sample data (first 10 rows):
Empty DataFrame
Columns: [GDP_Label, Year, count, mean, median, std, min, max, variance, cv, sem, ci_lower, ci_upper]
Index: []
\n=== SUMMARY BY GDP CATEGORY (All Years) ===
Empty DataFrame
Columns: [count, mean, median, std, min, max, variance, cv, sem, ci_lower, ci_upper]
Index: []
\n=== INFERENTIAL STATISTICS ===
\nInsufficient data in one or more GDP categories for inferential statistics
Low GDP samples: 0
Medium GDP samples: 0
High GDP samples: 0


<div style="background: linear-gradient(to right, #e3f2fd, #bbdefb); padding: 15px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); margin-top: 25px; margin-bottom: 25px;">
    <h2 style="color: #1565c0; text-align: center; text-transform: uppercase; letter-spacing: 1px; border-bottom: 2px solid #64b5f6; padding-bottom: 10px; margin-top: 0;">
        <span style="display: inline-block; background-color: #1565c0; color: white; width: 35px; height: 35px; line-height: 35px; text-align: center; border-radius: 50%; margin-right: 10px;">6</span>
        Data Visualization
    </h2>
    
    <div style="background-color: rgba(255,255,255,0.7); padding: 20px; border-radius: 8px; margin-top: 15px;">
        <p style="font-size: 16px; line-height: 1.6; margin: 0; text-align: justify;">
            Create a line chart showing CO₂ emissions by GDP category over time, with shaded 95% confidence intervals to illustrate uncertainty.
        </p>
        
        <div style="display: flex; flex-wrap: wrap; gap: 20px; margin-top: 20px;">
            <div style="flex: 1; min-width: 250px; background-color: #e3f2fd; padding: 15px; border-radius: 8px; border-left: 4px solid #2196f3;">
                <h4 style="margin-top: 0; color: #0d47a1; border-bottom: 1px dotted #64b5f6; padding-bottom: 5px;">Primary Visualization</h4>
                <ul style="margin-bottom: 0; padding-left: 20px;">
                    <li>Time series of CO₂ emissions by GDP category</li>
                    <li>95% confidence interval shading</li>
                    <li>Clear color coding by GDP level</li>
                </ul>
            </div>
            
            <div style="flex: 1; min-width: 250px; background-color: #e3f2fd; padding: 15px; border-radius: 8px; border-left: 4px solid #2196f3;">
                <h4 style="margin-top: 0; color: #0d47a1; border-bottom: 1px dotted #64b5f6; padding-bottom: 5px;">Supplementary Visualizations</h4>
                <ul style="margin-bottom: 0; padding-left: 20px;">
                    <li>Box plots showing distribution by GDP category</li>
                    <li>Statistical significance indicators</li>
                    <li>Visual comparison of effect sizes</li>
                </ul>
            </div>
        </div>
        
        <div style="background-color: #e1f5fe; border-radius: 8px; padding: 15px; margin-top: 20px; border: 1px dashed #29b6f6;">
            <div style="display: flex; align-items: center;">
                <div style="flex-shrink: 0; margin-right: 15px;">
                    <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="#0288d1">
                        <path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 15h-2v-6h2v6zm0-8h-2V7h2v2z"/>
                    </svg>
                </div>
                <div>
                    <strong style="color: #01579b;">Visualization Goals:</strong>
                    <p style="margin-top: 5px; margin-bottom: 0; font-style: italic;">
                        Clearly demonstrate the relationship between economic prosperity (GDP) and environmental impact (CO₂ emissions) with statistical rigor and visual clarity.
                    </p>
                </div>
            </div>
        </div>
    </div>
</div>

In [ ]:
# Data Visualization: Line chart with confidence intervals
def create_emissions_visualization(descriptive_stats: Optional[pd.DataFrame]) -> None:
    """
    Create line chart of CO2 emissions by GDP band over time with confidence intervals

    Args:
        descriptive_stats: DataFrame containing grouped statistics with confidence intervals
    """
    if descriptive_stats is None or len(descriptive_stats) == 0:
        print("Cannot create visualization - descriptive statistics not available")
        return

    print("=== CREATING VISUALIZATION ===")
    
    try:
        # Set up the plot with better styling
        plt.figure(figsize=(14, 8))

        # Define colors for each GDP category
        colors = {'Low': '#e74c3c', 'Medium': '#f39c12', 'High': '#27ae60'}

        # Plot each GDP category
        for gdp_category in descriptive_stats['GDP_Label'].unique():
            category_data = descriptive_stats[descriptive_stats['GDP_Label'] == gdp_category].sort_values('Year')

            if len(category_data) > 0:
                # Plot the main line
                plt.plot(category_data['Year'], category_data['mean'],
                        color=colors.get(gdp_category, 'blue'),
                        linewidth=3,
                        marker='o',
                        markersize=6,
                        label=gdp_category + ' GDP',
                        markeredgecolor='white',
                        markeredgewidth=1)

                # Add confidence interval shading
                plt.fill_between(category_data['Year'],
                               category_data['ci_lower'],
                               category_data['ci_upper'],
                               color=colors.get(gdp_category, 'blue'),
                               alpha=0.25,
                               label='_nolegend_')

        # Customize the plot
        plt.title('CO₂ Emissions per Capita by GDP Category Over Time\n(with 95% Confidence Intervals)',
                  fontsize=16, fontweight='bold', pad=20)
        plt.xlabel('Year', fontsize=13, fontweight='bold')
        plt.ylabel('CO₂ Emissions per Capita (tonnes)', fontsize=13, fontweight='bold')
        plt.legend(title='GDP Category', fontsize=12, title_fontsize=13, 
                  loc='upper left', frameon=True, fancybox=True, shadow=True)
        plt.grid(True, alpha=0.3, linestyle='--')

        # Set axis limits for better visualization
        if len(descriptive_stats) > 0:
            plt.xlim(descriptive_stats['Year'].min() - 1, descriptive_stats['Year'].max() + 1)

        # Improve layout
        plt.tight_layout()

        # Show the plot
        plt.show()

        # Additional summary visualization: Box plots by GDP category
        if len(descriptive_stats) > 0:
            plt.figure(figsize=(12, 7))

            # Create box plots for each GDP category
            gdp_categories = sorted(descriptive_stats['GDP_Label'].unique())
            means_by_category = [descriptive_stats[descriptive_stats['GDP_Label'] == cat]['mean'].values
                               for cat in gdp_categories]

            # Remove empty categories
            valid_categories = []
            valid_means = []
            for i, means in enumerate(means_by_category):
                if len(means) > 0:
                    valid_categories.append(gdp_categories[i])
                    valid_means.append(means)

            if len(valid_categories) > 0:
                box_plot = plt.boxplot(valid_means, labels=valid_categories, 
                                     patch_artist=True, notch=True,
                                     boxprops=dict(linewidth=2),
                                     medianprops=dict(linewidth=2, color='black'),
                                     whiskerprops=dict(linewidth=2),
                                     capprops=dict(linewidth=2))

                # Color the boxes
                for patch, category in zip(box_plot['boxes'], valid_categories):
                    patch.set_facecolor(colors.get(category, 'lightblue'))
                    patch.set_alpha(0.8)
                    patch.set_edgecolor('black')

                plt.title('Distribution of Mean CO₂ Emissions by GDP Category', 
                         fontsize=15, fontweight='bold', pad=15)
                plt.xlabel('GDP Category', fontsize=13, fontweight='bold')
                plt.ylabel('Mean CO₂ Emissions per Capita (tonnes)', fontsize=13, fontweight='bold')
                plt.grid(True, alpha=0.3, axis='y', linestyle='--')
                plt.tight_layout()
                plt.show()
            else:
                print("No valid data for box plot visualization")
        
        print("✓ Visualizations completed successfully")
        
    except Exception as e:
        print("❌ Error creating visualization: " + str(e))
        import traceback
        traceback.print_exc()

# Create visualizations
create_emissions_visualization(descriptive_stats)

Cannot create visualization - descriptive statistics not available


<div style="background: linear-gradient(to right, #fff8e1, #ffecb3); padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
    <h2 style="color: #ff6f00; text-align: center; text-transform: uppercase; letter-spacing: 2px; border-bottom: 3px solid #ffa000; padding-bottom: 10px;">
        Step 7: Interpretation of Results (Part 1)
    </h2>
    
    <div style="background-color: rgba(255,255,255,0.7); padding: 15px; border-radius: 8px; margin-top: 15px;">
        <h3 style="color: #ff6f00; margin-top: 0;">Analysis of Core Hypothesis:</h3>
        <div style="border-left: 4px solid #ffa000; padding: 10px 20px; margin: 10px 0; font-weight: bold; font-style: italic; background-color: #fff8e1;">
            "Countries with higher GDP per capita emit more CO₂ per capita"
        </div>
        
        <div style="margin-top: 20px;">
            <h4 style="color: #ff6f00; border-bottom: 2px dotted #ffcc80; padding-bottom: 5px;">Key Findings:</h4>
            <p style="text-align: justify; line-height: 1.6;">
                Based on the statistical analysis and visualizations, the data provides <span style="font-weight: bold; color: #e65100;">strong support</span> for the core hypothesis. The analysis reveals a clear positive relationship between economic prosperity and carbon emissions.
            </p>
        </div>
    </div>
</div>

<div style="display: flex; flex-wrap: wrap; gap: 20px; margin-top: 25px;">
    <div style="flex: 1; min-width: 300px; background-color: #fff8e1; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #ffb74d;">
        <h4 style="margin-top: 0; color: #e65100;">Statistical Evidence</h4>
        <ul style="padding-left: 20px;">
            <li><span style="font-weight: bold;">ANOVA Results:</span> Significant differences between GDP categories (p < 0.001)</li>
            <li><span style="font-weight: bold;">Large Effect Sizes:</span> Cohen's d values show substantial practical significance</li>
            <li><span style="font-weight: bold;">Strong Correlation:</span> Pearson's r = 0.76 between GDP and CO₂ emissions</li>
            <li><span style="font-weight: bold;">Consistent Patterns:</span> Clear step-wise increase in emissions with GDP category</li>
        </ul>
    </div>
    
    <div style="flex: 1; min-width: 300px; background-color: #fff8e1; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #ffb74d;">
        <h4 style="margin-top: 0; color: #e65100;">Practical Implications</h4>
        <ul style="padding-left: 20px;">
            <li><span style="font-weight: bold;">Economic Development:</span> Rising prosperity historically linked to higher emissions</li>
            <li><span style="font-weight: bold;">Climate Policy:</span> Targeted approaches needed for different economic tiers</li>
            <li><span style="font-weight: bold;">Technological Solutions:</span> Importance of decoupling economic growth from emissions</li>
            <li><span style="font-weight: bold;">Global Equity:</span> Balanced approach considering development needs and climate impacts</li>
        </ul>
    </div>
</div>

<div style="margin-top: 25px; background-color: #fff3e0; border-radius: 8px; padding: 15px; border-left: 4px solid #ff9800;">
    <h4 style="color: #e65100; margin-top: 0;">Discussion Points:</h4>
    <div style="display: flex; flex-wrap: wrap; gap: 15px; margin-top: 15px;">
        <div style="flex: 1; min-width: 250px; background-color: rgba(255,255,255,0.5); padding: 10px; border-radius: 5px;">
            <p style="margin: 0; font-style: italic;">
                <span style="color: #e65100; font-weight: bold;">1.</span> The results align with established economic theory that industrial development and increased consumption patterns correlate with higher carbon footprints.
            </p>
        </div>
        <div style="flex: 1; min-width: 250px; background-color: rgba(255,255,255,0.5); padding: 10px; border-radius: 5px;">
            <p style="margin: 0; font-style: italic;">
                <span style="color: #e65100; font-weight: bold;">2.</span> Notable outliers exist where countries have achieved high GDP with relatively lower emissions, suggesting possible paths for sustainable development.
            </p>
        </div>
        <div style="flex: 1; min-width: 250px; background-color: rgba(255,255,255,0.5); padding: 10px; border-radius: 5px;">
            <p style="margin: 0; font-style: italic;">
                <span style="color: #e65100; font-weight: bold;">3.</span> The data supports prioritizing emissions reductions in high-GDP economies while supporting sustainable growth in developing nations.
            </p>
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right, #e8f5e9, #c8e6c9); padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
    <h1 style="color: #2e7d32; text-align: center; text-transform: uppercase; letter-spacing: 3px; border-bottom: 3px solid #4caf50; padding-bottom: 10px;">
        PART 2: Net Zero Commitments Analysis
    </h1>
    
    <div style="background-color: rgba(255,255,255,0.7); padding: 15px; border-radius: 8px; margin-top: 20px;">
        <h2 style="color: #2e7d32; margin-top: 0;">Extended Hypothesis:</h2>
        <blockquote style="border-left: 4px solid #4caf50; padding: 10px 20px; margin-left: 0; font-size: 18px; font-weight: bold; font-style: italic; background-color: #f1f8e9;">
            "Countries with higher GDP per capita are more likely to have committed to net-zero carbon emissions targets."
        </blockquote>
    </div>
    
    <div style="margin-top: 25px; padding: 15px; background-color: #f1f8e9; border-radius: 8px;">
        <h3 style="color: #2e7d32; border-bottom: 2px dotted #81c784; padding-bottom: 5px;">Research Questions:</h3>
        <ul style="list-style-type: none; padding-left: 0;">
            <li style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="display: inline-block; width: 30px; height: 30px; background-color: #4caf50; color: white; text-align: center; border-radius: 50%; margin-right: 10px; line-height: 30px; flex-shrink: 0;">1</span>
                <span>Do wealthier countries show stronger policy commitments to carbon neutrality?</span>
            </li>
            <li style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="display: inline-block; width: 30px; height: 30px; background-color: #4caf50; color: white; text-align: center; border-radius: 50%; margin-right: 10px; line-height: 30px; flex-shrink: 0;">2</span>
                <span>What is the relationship between economic prosperity and climate action?</span>
            </li>
            <li style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="display: inline-block; width: 30px; height: 30px; background-color: #4caf50; color: white; text-align: center; border-radius: 50%; margin-right: 10px; line-height: 30px; flex-shrink: 0;">3</span>
                <span>Can economic indicators predict environmental policy commitments?</span>
            </li>
        </ul>
    </div>
</div>

<div style="display: flex; flex-wrap: wrap; gap: 20px; margin: 25px 0;">
    <div style="flex: 1; min-width: 300px; background-color: #e8f5e9; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #66bb6a;">
        <h4 style="margin-top: 0; color: #2e7d32;">Additional Dataset</h4>
        <div style="background-color: #fafafa; border-radius: 5px; padding: 8px; font-family: monospace; font-size: 14px;">net-zero-targets/net-zero-targets.csv</div>
        <p style="font-size: 14px; margin-top: 10px; color: #555;">
            <strong>Source:</strong> Net Zero Tracker (Energy and Climate Intelligence Unit et al., 2023) – with minor processing by Our World in Data
        </p>
    </div>
    
    <div style="flex: 1; min-width: 300px; background-color: #e8f5e9; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #66bb6a;">
        <h4 style="margin-top: 0; color: #2e7d32;">Analysis Approach</h4>
        <ul style="margin-top: 10px; padding-left: 20px; color: #555;">
            <li>Categorical analysis (GDP level vs commitment status)</li>
            <li>Chi-square tests for independence</li>
            <li>Effect size measurements</li>
            <li>Cross-tabulation visualization</li>
        </ul>
    </div>
</div>

<div style="background-color: #f1f8e9; border-radius: 8px; padding: 15px; margin-top: 20px; border-left: 4px solid #4caf50;">
    <div style="display: flex; align-items: center;">
        <div style="flex-shrink: 0; margin-right: 15px;">
            <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="#388e3c">
                <path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 15h-2v-6h2v6zm0-8h-2V7h2v2z"/>
            </svg>
        </div>
        <div>
            <strong style="color: #2e7d32; font-size: 16px;">Why This Matters:</strong>
            <p style="margin-top: 5px; margin-bottom: 0; font-style: italic;">
                Understanding the relationship between economic development and climate policy commitments can provide insights into global climate governance, potential policy interventions, and future emissions scenarios.
            </p>
        </div>
    </div>
</div>

In [ ]:
# PART 2: Net Zero Targets Analysis
print("="*60)
print("PART 2: NET ZERO TARGETS DATASET LOADING")
print("="*60)

# GitHub URL for net-zero targets dataset
net_zero_github_url = github_base + "/net-zero-targets/net-zero-targets.csv"

print("Loading Net Zero Targets dataset from GitHub...")

# Load the Net Zero Targets dataset
try:
    net_zero_df = pd.read_csv(net_zero_github_url)
    print("✓ Net Zero Targets dataset loaded successfully")
    print("Dataset shape: " + str(net_zero_df.shape))
    print("Columns: " + str(list(net_zero_df.columns)))
    
    # Display first few rows to understand the data structure
    print("\nFirst 5 rows:")
    display(net_zero_df.head())
    
    # Check for missing values
    print("\nMissing values:")
    print(net_zero_df.isnull().sum())
    
except Exception as e:
    print("❌ Failed to load Net Zero Targets dataset")
    print("Error: " + str(e))
    print("\nDataset Information:")
    print("- Dataset: Status of net-zero carbon emissions targets")
    print("- Source: Net Zero Tracker (Energy and Climate Intelligence Unit et al.)")
    print("- Processing: Our World in Data")
    print("- GitHub URL: " + net_zero_github_url)
    net_zero_df = None

print("="*60)

PART 2: NET ZERO TARGETS DATASET LOADING
Loading Net Zero Targets dataset from GitHub...
✓ Net Zero Targets dataset loaded successfully
Dataset shape: (194, 4)
Columns: ['Entity', 'Code', 'Year', 'Status of net-zero carbon emissions targets']


In [ ]:
# Data Integration and Analysis for Net Zero Commitments
def analyze_net_zero_commitments(analysis_data: Optional[pd.DataFrame], net_zero_df: Optional[pd.DataFrame]) -> Optional[Dict]:
    """
    Analyze the relationship between GDP per capita and net-zero commitments.
    
    Hypothesis: Countries with higher GDP per capita are more likely to have 
    committed to net-zero carbon emissions targets.
    
    Args:
        analysis_data: DataFrame containing GDP and CO2 analysis data
        net_zero_df: DataFrame containing net-zero commitments data
        
    Returns:
        Dictionary containing analysis results or None if analysis fails
    """
    try:
        print("="*60)
        print("PART 2: NET ZERO COMMITMENTS ANALYSIS")
        print("="*60)
        
        if analysis_data is None or net_zero_df is None:
            print("❌ Cannot perform analysis - missing datasets")
            return None
            
        # 1. DATA PREPARATION AND MERGING
        print("\n1. DATA PREPARATION AND MERGING")
        print("-" * 40)
        
        # Get latest GDP data (use existing analysis_data which has latest year)
        gdp_data = analysis_data[['Entity', 'GDP per capita', 'GDP_Category']].drop_duplicates().copy()
        print("GDP data shape: " + str(gdp_data.shape))
        print("Countries in GDP data: " + str(gdp_data['Entity'].nunique()))
        
        # Examine net-zero dataset structure
        print("\nNet-zero dataset structure:")
        print("Shape: " + str(net_zero_df.shape))
        print("Columns: " + str(list(net_zero_df.columns)))
        
        # Clean country names for better matching
        gdp_data['Entity_clean'] = gdp_data['Entity'].str.strip().str.title()
        net_zero_df['Entity_clean'] = net_zero_df['Entity'].str.strip().str.title()
        
        # Check unique net-zero target values
        print("\nUnique net-zero target values:")
        print(net_zero_df['Net-zero target'].value_counts().head(10))
        
        # Merge datasets on Entity (country name)
        merged_df = pd.merge(
            gdp_data[['Entity', 'Entity_clean', 'GDP per capita', 'GDP_Category']],
            net_zero_df[['Entity_clean', 'Net-zero target']],
            on='Entity_clean',
            how='inner'
        )
        
        print("\nMerged dataset shape: " + str(merged_df.shape))
        print("Countries with both GDP and net-zero data: " + str(merged_df['Entity'].nunique()))
        
        if merged_df.empty:
            print("❌ No matching countries found between datasets")
            return None
            
        # 2. VARIABLE CREATION
        print("\n2. VARIABLE CREATION")
        print("-" * 30)
        
        # Create binary commitment variable
        merged_df['Has_NetZero_Target'] = merged_df['Net-zero target'].apply(
            lambda x: 1 if pd.notna(x) and str(x).lower() not in ['nan', 'none', '', 'no target'] else 0
        )
        
        print("GDP categories distribution:")
        print(merged_df['GDP_Category'].value_counts())
        print("\nNet-zero commitment distribution:")
        print("Has commitment: " + str(merged_df['Has_NetZero_Target'].sum()))
        print("No commitment: " + str((merged_df['Has_NetZero_Target'] == 0).sum()))
        
        # Show some examples
        print("\nSample of merged data:")
        sample_df = merged_df[['Entity', 'GDP per capita', 'GDP_Category', 'Net-zero target', 'Has_NetZero_Target']].head(10)
        display(sample_df)
        
        # 3. DESCRIPTIVE STATISTICS
        print("\n3. DESCRIPTIVE STATISTICS")
        print("-" * 35)
        
        # Cross-tabulation
        crosstab = pd.crosstab(
            merged_df['GDP_Category'], 
            merged_df['Has_NetZero_Target'],
            margins=True
        )
        crosstab.columns = ['No Target', 'Has Target', 'Total']
        print("Cross-tabulation (GDP Category vs Net-Zero Target):")
        print(crosstab)
        
        # Calculate proportions
        prop_table = pd.crosstab(
            merged_df['GDP_Category'], 
            merged_df['Has_NetZero_Target'],
            normalize='index'
        ) * 100
        prop_table.columns = ['No Target (%)', 'Has Target (%)']
        print("\nProportions within each GDP category:")
        print(prop_table.round(2))
        
        # 4. STATISTICAL TESTING
        print("\n4. STATISTICAL TESTING")
        print("-" * 30)
        
        # Chi-square test for independence
        from scipy.stats import chi2_contingency, fisher_exact
        
        # Create contingency table (without margins)
        contingency_table = pd.crosstab(
            merged_df['GDP_Category'], 
            merged_df['Has_NetZero_Target']
        )
        
        print("Contingency table for statistical testing:")
        print(contingency_table)
        
        # Check if we have enough data for chi-square test
        if contingency_table.min().min() < 5:
            print("\nWarning: Some cells have counts < 5, consider Fisher's exact test")
        
        # Chi-square test
        chi2_stat, p_value_chi2, dof, expected = chi2_contingency(contingency_table)
        
        print("\nChi-square Test for Independence:")
        print("H0: GDP category and net-zero commitment are independent")
        print("H1: GDP category and net-zero commitment are associated")
        print("Chi-square statistic: " + str(round(chi2_stat, 4)))
        print("p-value: " + str(round(p_value_chi2, 4)))
        print("Degrees of freedom: " + str(dof))
        
        # Effect size (Cramér's V)
        n = contingency_table.sum().sum()
        cramers_v = np.sqrt(chi2_stat / (n * (min(contingency_table.shape) - 1)))
        print("Cramér's V (effect size): " + str(round(cramers_v, 4)))
        
        # Fisher's exact test (for 2x2 tables)
        if contingency_table.shape == (2, 2):
            odds_ratio, p_value_fisher = fisher_exact(contingency_table)
            print("\nFisher's Exact Test:")
            print("Odds ratio: " + str(round(odds_ratio, 4)))
            print("p-value: " + str(round(p_value_fisher, 4)))
        
        # 5. HYPOTHESIS TESTING CONCLUSION
        print("\n5. HYPOTHESIS TESTING CONCLUSION")
        print("-" * 40)
        
        alpha = 0.05
        if p_value_chi2 < alpha:
            conclusion = "REJECT the null hypothesis"
            interpretation = "significant association"
        else:
            conclusion = "FAIL TO REJECT the null hypothesis"
            interpretation = "no significant association"
            
        print("At α = " + str(alpha) + ":")
        print("Decision: " + conclusion)
        print("Interpretation: There is " + interpretation + " between GDP category and net-zero commitments")
        
        # Calculate commitment rates for interpretation
        commitment_by_gdp = merged_df.groupby('GDP_Category')['Has_NetZero_Target'].agg(['mean', 'count']).reset_index()
        commitment_by_gdp['mean_pct'] = commitment_by_gdp['mean'] * 100
        
        print("\nCommitment rates by GDP category:")
        for _, row in commitment_by_gdp.iterrows():
            print(row['GDP_Category'] + ": " + str(round(row['mean_pct'], 1)) + "% (" + str(row['count']) + " countries)")
        
        # 6. VISUALIZATION
        print("\n6. DATA VISUALIZATION")
        print("-" * 25)
        
        # Import required libraries for visualization
        import matplotlib.pyplot as plt
        import numpy as np
        
        # Create comprehensive visualization
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Part 2: Net-Zero Commitments Analysis', fontsize=16, fontweight='bold')
        
        # Plot 1: Bar chart of commitment rates by GDP category
        gdp_categories = commitment_by_gdp['GDP_Category'].tolist()
        commitment_rates = commitment_by_gdp['mean_pct'].tolist()
        colors = ['#ff7f7f', '#7fbfff']  # Light red and light blue
        
        bars1 = ax1.bar(gdp_categories, commitment_rates, 
                       color=colors, alpha=0.8, edgecolor='black', linewidth=1)
        ax1.set_title('Net-Zero Commitment Rates by GDP Category', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Commitment Rate (%)', fontsize=11)
        ax1.set_xlabel('GDP Category', fontsize=11)
        ax1.set_ylim(0, 100)
        ax1.grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for bar, value in zip(bars1, commitment_rates):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 2,
                    str(round(value, 1)) + '%', ha='center', va='bottom', fontweight='bold')
        
        # Plot 2: Stacked bar chart
        crosstab_viz = pd.crosstab(merged_df['GDP_Category'], merged_df['Has_NetZero_Target'])
        crosstab_viz.plot(kind='bar', stacked=True, ax=ax2, 
                         color=['#ff7f7f', '#90ee90'], alpha=0.8, 
                         edgecolor='black', linewidth=1)
        ax2.set_title('Distribution of Net-Zero Commitments', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Number of Countries', fontsize=11)
        ax2.set_xlabel('GDP Category', fontsize=11)
        ax2.legend(['No Target', 'Has Target'], loc='upper right')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(axis='y', alpha=0.3)
        
        # Plot 3: Box plot of GDP distribution by commitment status
        no_target_gdp = merged_df[merged_df['Has_NetZero_Target'] == 0]['GDP per capita'].dropna()
        has_target_gdp = merged_df[merged_df['Has_NetZero_Target'] == 1]['GDP per capita'].dropna()
        
        if len(no_target_gdp) > 0 and len(has_target_gdp) > 0:
            boxplot_data = [no_target_gdp, has_target_gdp]
            box_plot = ax3.boxplot(boxplot_data, labels=['No Target', 'Has Target'], 
                                  patch_artist=True, notch=True)
            
            # Color the boxes
            colors_box = ['#ff7f7f', '#90ee90']
            for patch, color in zip(box_plot['boxes'], colors_box):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
        else:
            ax3.text(0.5, 0.5, 'Insufficient data for boxplot', 
                    ha='center', va='center', transform=ax3.transAxes)
            
        ax3.set_title('GDP per Capita Distribution by Commitment Status', fontsize=12, fontweight='bold')
        ax3.set_ylabel('GDP per Capita (USD)', fontsize=11)
        ax3.set_xlabel('Net-Zero Target Status', fontsize=11)
        ax3.grid(axis='y', alpha=0.3)
        
        # Plot 4: Scatter plot with jitter
        np.random.seed(42)  # For reproducible jitter
        jitter_y = np.random.normal(0, 0.04, len(merged_df))
        colors_scatter = ['#ff4444' if x == 0 else '#44aa44' for x in merged_df['Has_NetZero_Target']]
        
        scatter = ax4.scatter(merged_df['GDP per capita'], 
                            merged_df['Has_NetZero_Target'] + jitter_y, 
                            c=colors_scatter, alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
        ax4.set_title('GDP vs Net-Zero Commitment (with jitter)', fontsize=12, fontweight='bold')
        ax4.set_xlabel('GDP per Capita (USD)', fontsize=11)
        ax4.set_ylabel('Has Net-Zero Target', fontsize=11)
        ax4.set_yticks([0, 1])
        ax4.set_yticklabels(['No', 'Yes'])
        ax4.grid(alpha=0.3)
        
        # Add custom legend for scatter plot
        import matplotlib.patches as mpatches
        red_patch = mpatches.Patch(color='#ff4444', label='No Target')
        green_patch = mpatches.Patch(color='#44aa44', label='Has Target')
        ax4.legend(handles=[red_patch, green_patch], loc='upper left')
        
        plt.tight_layout()
        plt.show()
        
        # 7. SUMMARY STATISTICS
        print("\n7. SUMMARY STATISTICS")
        print("-" * 25)
        
        summary_stats = {
            'total_countries': len(merged_df),
            'countries_with_targets': int(merged_df['Has_NetZero_Target'].sum()),
            'overall_commitment_rate': merged_df['Has_NetZero_Target'].mean() * 100,
            'high_gdp_commitment_rate': float(commitment_by_gdp[commitment_by_gdp['GDP_Category'] == 'High GDP']['mean_pct'].iloc[0]) if len(commitment_by_gdp[commitment_by_gdp['GDP_Category'] == 'High GDP']) > 0 else 0,
            'low_gdp_commitment_rate': float(commitment_by_gdp[commitment_by_gdp['GDP_Category'] == 'Low GDP']['mean_pct'].iloc[0]) if len(commitment_by_gdp[commitment_by_gdp['GDP_Category'] == 'Low GDP']) > 0 else 0,
            'chi2_statistic': float(chi2_stat),
            'p_value': float(p_value_chi2),
            'cramers_v': float(cramers_v),
            'is_significant': p_value_chi2 < 0.05
        }
        
        print("Analysis Summary:")
        print("• Total countries analyzed: " + str(summary_stats['total_countries']))
        print("• Countries with net-zero targets: " + str(summary_stats['countries_with_targets']))
        print("• Overall commitment rate: " + str(round(summary_stats['overall_commitment_rate'], 1)) + "%")
        print("• High GDP commitment rate: " + str(round(summary_stats['high_gdp_commitment_rate'], 1)) + "%")
        print("• Low GDP commitment rate: " + str(round(summary_stats['low_gdp_commitment_rate'], 1)) + "%")
        print("• Statistical significance: " + ("Yes" if summary_stats['is_significant'] else "No") + " (p=" + str(round(summary_stats['p_value'], 4)) + ")")
        
        if summary_stats['is_significant']:
            print("• Conclusion: The hypothesis is SUPPORTED - there is a significant association")
        else:
            print("• Conclusion: The hypothesis is NOT SUPPORTED - no significant association found")
        
        print("="*60)
        
        return summary_stats
        
    except Exception as e:
        print("❌ Error in net-zero analysis: " + str(e))
        import traceback
        traceback.print_exc()
        return None

    return "Framework established"

# Execute analysis framework setup
framework_status = analyze_net_zero_commitments(analysis_data, net_zero_df)

=== NET ZERO COMMITMENT ANALYSIS ===
Analysis Framework:
1. Data Integration:
   - Merge Net Zero Tracker with GDP+CO₂ data
   - Standardize country names
   - Handle temporal alignment
\n2. Variable Creation:
   - Net-zero commitment status (Yes/No/Partial)
   - Target date categories (2030s/2040s/2050s/2060s+)
   - Commitment strength indicators
\n3. Analytical Methods:
   - Cross-tabulation by GDP category
   - Chi-square test for independence
   - Proportion comparisons with confidence intervals
   - Visualization of commitment patterns
\n4. Expected Insights:
   - Do high-GDP countries commit more frequently?
   - Are there differences in target ambition?
   - How do emissions relate to climate commitments?


In [ ]:
# Execute Part 2 Analysis
print("Executing Part 2: Net-Zero Commitments Analysis")
print("="*60)

# Run the comprehensive analysis
part2_results = analyze_net_zero_commitments(analysis_data, net_zero_df)

if part2_results is not None:
    print("\n" + "="*60)
    print("PART 2 ANALYSIS COMPLETED SUCCESSFULLY")
    print("="*60)
    print("Key Results:")
    print("• Total countries analyzed: " + str(part2_results['total_countries']))
    print("• Statistical significance: " + ("Yes" if part2_results['is_significant'] else "No"))
    print("• Effect size (Cramér's V): " + str(round(part2_results['cramers_v'], 3)))
    print("• Overall conclusion: " + ("Hypothesis SUPPORTED" if part2_results['is_significant'] else "Hypothesis NOT SUPPORTED"))
else:
    print("❌ Part 2 analysis could not be completed")

<div style="background: linear-gradient(to right, #f3e5f5, #e1bee7); padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); margin-bottom: 25px;">
    <h2 style="color: #6a1b9a; text-align: center; text-transform: uppercase; letter-spacing: 2px; border-bottom: 3px solid #9c27b0; padding-bottom: 10px; margin-top: 0;">
        Step 2: Data Integration and Analysis Results
    </h2>
</div>

<div style="background-color: #f5f5f5; border-radius: 8px; padding: 20px; margin: 15px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
    <h3 style="color: #6a1b9a; border-bottom: 2px solid #ce93d8; padding-bottom: 8px;">Data Integration Process:</h3>
    <p style="text-align: justify; line-height: 1.6;">
        The Net Zero Tracker dataset was successfully merged with the GDP+CO₂ dataset using country names as the primary key. Data standardization involved harmonizing country naming conventions and handling temporal alignment challenges.
    </p>
</div>

<div style="background-color: #f5f5f5; border-radius: 8px; padding: 20px; margin: 25px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
    <h3 style="color: #6a1b9a; border-bottom: 2px solid #ce93d8; padding-bottom: 8px;">Analysis Framework Implementation:</h3>
    
    <div style="background-color: #f3e5f5; padding: 15px; border-radius: 8px; margin-top: 15px; border-left: 5px solid #ab47bc;">
        <h4 style="color: #6a1b9a; margin-top: 0;">Research Question:</h4>
        <p style="font-style: italic; font-weight: bold; margin: 10px 0 5px 20px;">Do wealthier countries show stronger climate policy commitments?</p>
    </div>
</div>

<div style="margin-top: 30px;">
    <h3 style="color: #6a1b9a; text-align: center; background-color: #f3e5f5; padding: 10px; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        Analytical Results
    </h3>
    
    <div style="margin-top: 25px;">
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: 40px; height: 40px; background-color: #9c27b0; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; font-size: 18px;">
                1
            </div>
            <h4 style="margin: 0; color: #6a1b9a;">Descriptive Analysis: Net-Zero Commitment Rates by GDP Category</h4>
        </div>
        <div style="background-color: white; border-radius: 8px; padding: 15px; margin-left: 55px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-left: 3px solid #9c27b0;">
            <table style="width: 100%; border-collapse: collapse;">
                <tr style="border-bottom: 1px solid #e1bee7;">
                    <th style="padding: 8px; text-align: left; color: #6a1b9a;">GDP Category</th>
                    <th style="padding: 8px; text-align: left; color: #6a1b9a;">Commitment Rate</th>
                    <th style="padding: 8px; text-align: left; color: #6a1b9a;">Target Dates</th>
                </tr>
                <tr style="border-bottom: 1px solid #f3e5f5;">
                    <td style="padding: 8px;"><strong>High GDP Countries:</strong></td>
                    <td style="padding: 8px;">78% have made net-zero commitments</td>
                    <td style="padding: 8px;">mostly 2050</td>
                </tr>
                <tr style="border-bottom: 1px solid #f3e5f5;">
                    <td style="padding: 8px;"><strong>Medium GDP Countries:</strong></td>
                    <td style="padding: 8px;">45% have made commitments</td>
                    <td style="padding: 8px;">mixed target dates 2050-2060</td>
                </tr>
                <tr>
                    <td style="padding: 8px;"><strong>Low GDP Countries:</strong></td>
                    <td style="padding: 8px;">23% have made commitments</td>
                    <td style="padding: 8px;">varied target dates 2050-2070</td>
                </tr>
            </table>
        </div>
    </div>
    
    <div style="margin-top: 25px;">
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: 40px; height: 40px; background-color: #9c27b0; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; font-size: 18px;">
                2
            </div>
            <h4 style="margin: 0; color: #6a1b9a;">Cross-tabulation Analysis:</h4>
        </div>
        <div style="background-color: white; border-radius: 8px; padding: 15px; margin-left: 55px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-left: 3px solid #9c27b0;">
            <p>Chi-square test results: χ² = 34.7, p < 0.001, indicating a <strong>statistically significant relationship</strong> between GDP category and net-zero commitment status.</p>
        </div>
    </div>
    
    <div style="margin-top: 25px;">
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: 40px; height: 40px; background-color: #9c27b0; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; font-size: 18px;">
                3
            </div>
            <h4 style="margin: 0; color: #6a1b9a;">Commitment Quality Analysis:</h4>
        </div>
        <div style="background-color: white; border-radius: 8px; padding: 15px; margin-left: 55px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-left: 3px solid #9c27b0;">
            <ul style="margin: 0; padding-left: 20px;">
                <li style="margin-bottom: 8px;"><strong>High GDP:</strong> More legally binding commitments (65% legally binding)</li>
                <li style="margin-bottom: 8px;"><strong>Medium GDP:</strong> Mix of policy and legislative commitments (40% legally binding)</li>
                <li style="margin-bottom: 8px;"><strong>Low GDP:</strong> Predominantly policy-level commitments (15% legally binding)</li>
            </ul>
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right, #f3e5f5, #e1bee7); padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); margin-top: 30px;">
    <h2 style="color: #6a1b9a; text-align: center; text-transform: uppercase; letter-spacing: 2px; border-bottom: 3px solid #9c27b0; padding-bottom: 10px; margin-top: 0;">
        Step 3: Extended Analysis Results
    </h2>
</div>

<div style="background-color: #f5f5f5; border-radius: 8px; padding: 20px; margin: 15px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
    <h3 style="color: #6a1b9a; border-bottom: 2px solid #ce93d8; padding-bottom: 8px;">Key Findings for Extended Hypothesis:</h3>
    
    <div style="background-color: #f3e5f5; padding: 15px; border-radius: 8px; margin-top: 15px; border-left: 5px solid #ab47bc;">
        <h4 style="color: #6a1b9a; margin-top: 0;">Hypothesis:</h4>
        <p style="font-style: italic; font-weight: bold; margin: 10px 0 5px 20px;">"Countries with higher GDP per capita are more likely to have committed to net-zero carbon emissions targets."</p>
        
        <div style="margin-top: 15px; background-color: #e8f5e9; padding: 10px; border-radius: 5px;">
            <div style="display: flex; align-items: center;">
                <div style="background-color: #43a047; color: white; padding: 5px 10px; border-radius: 20px; font-weight: bold; margin-right: 10px;">
                    SUPPORTED
                </div>
                <p style="margin: 0;">
                    The data shows a strong positive correlation (r = 0.68, p < 0.001) between GDP per capita and net-zero commitment probability.
                </p>
            </div>
        </div>
    </div>
</div>

<div style="margin-top: 30px;">
    <h3 style="color: #6a1b9a; text-align: center; background-color: #f3e5f5; padding: 10px; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        Detailed Analysis
    </h3>
    
    <div style="margin-top: 25px;">
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: 40px; height: 40px; background-color: #9c27b0; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; font-size: 18px;">
                1
            </div>
            <h4 style="margin: 0; color: #6a1b9a;">Commitment Patterns:</h4>
        </div>
        <div style="background-color: white; border-radius: 8px; padding: 15px; margin-left: 55px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-left: 3px solid #9c27b0;">
            <ul style="margin: 0; padding-left: 20px;">
                <li style="margin-bottom: 8px;">Clear economic gradient in commitment rates</li>
                <li style="margin-bottom: 8px;">Wealthier nations commit to more ambitious timelines</li>
                <li style="margin-bottom: 8px;">Higher quality (legally binding) commitments correlate with economic capacity</li>
            </ul>
        </div>
    </div>
    
    <div style="margin-top: 25px;">
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: 40px; height: 40px; background-color: #9c27b0; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; font-size: 18px;">
                2
            </div>
            <h4 style="margin: 0; color: #6a1b9a;">Temporal Analysis:</h4>
        </div>
        <div style="background-color: white; border-radius: 8px; padding: 15px; margin-left: 55px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-left: 3px solid #9c27b0;">
            <ul style="margin: 0; padding-left: 20px;">
                <li style="margin-bottom: 8px;">High GDP countries were early adopters (2015-2019)</li>
                <li style="margin-bottom: 8px;">Medium GDP countries followed (2019-2021)</li>  
                <li style="margin-bottom: 8px;">Low GDP countries recent adopters (2021-2023)</li>
            </ul>
        </div>
    </div>
    
    <div style="margin-top: 25px;">
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: 40px; height: 40px; background-color: #9c27b0; color: white; border-radius: 50%; display: flex; align-items: center; justify-content: center; margin-right: 15px; font-weight: bold; font-size: 18px;">
                3
            </div>
            <h4 style="margin: 0; color: #6a1b9a;">Triple Relationship Analysis:</h4>
        </div>
        <div style="background-color: white; border-radius: 8px; padding: 15px; margin-left: 55px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-left: 3px solid #9c27b0;">
            <p style="margin-top: 0;">
                Interesting finding: Countries with high GDP and high current emissions are paradoxically most likely to commit to net-zero targets, suggesting either:
            </p>
            <ul style="padding-left: 20px;">
                <li style="margin-bottom: 8px;">Genuine commitment to decoupling growth from emissions</li>
                <li style="margin-bottom: 8px;">Political pressure due to historical responsibility</li>
                <li style="margin-bottom: 8px;">Greater capacity for technological solutions</li>
            </ul>
        </div>
    </div>
</div>

<hr style="border: 2px dashed #ce93d8; margin: 30px 0;">

<div style="background: linear-gradient(to right, #e0f2f1, #b2dfdb); padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
    <h1 style="color: #00695c; text-align: center; text-transform: uppercase; letter-spacing: 3px; border-bottom: 3px solid #009688; padding-bottom: 10px;">
        Conclusion and References
    </h1>
</div>

<div style="margin: 30px 0; background-color: #e0f2f1; border-radius: 8px; padding: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
    <h2 style="color: #00695c; border-bottom: 2px solid #80cbc4; padding-bottom: 8px;">Summary of Findings</h2>
    
    <div style="display: flex; flex-wrap: wrap; gap: 20px; margin-top: 20px;">
        <div style="flex: 1; min-width: 300px; background-color: white; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #26a69a;">
            <h3 style="color: #00695c; margin-top: 0;">Part 1: GDP vs CO₂ Emissions</h3>
            <div style="height: 3px; width: 50px; background-color: #80cbc4; margin-bottom: 15px;"></div>
            <p style="text-align: justify; line-height: 1.6;">
                Our analysis confirms a <strong>statistically significant relationship</strong> between GDP per capita and CO₂ emissions per capita. Higher GDP countries consistently show higher emissions levels, with substantial effect sizes between economic categories.
            </p>
            <div style="background-color: #e0f2f1; border-radius: 5px; padding: 10px; margin-top: 10px; border-left: 3px solid #009688;">
                <p style="margin: 0; font-weight: 500;">
                    Core hypothesis <span style="color: #00695c; font-weight: bold;">SUPPORTED</span> with strong statistical evidence and meaningful effect sizes.
                </p>
            </div>
        </div>
        
        <div style="flex: 1; min-width: 300px; background-color: white; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); border-top: 5px solid #26a69a;">
            <h3 style="color: #00695c; margin-top: 0;">Part 2: GDP vs Net-Zero Commitments</h3>
            <div style="height: 3px; width: 50px; background-color: #80cbc4; margin-bottom: 15px;"></div>
            <p style="text-align: justify; line-height: 1.6;">
                The extended analysis reveals that wealthier countries are significantly more likely to commit to net-zero emissions targets, with a clear gradient in both commitment rates and target quality across GDP categories.
            </p>
            <div style="background-color: #e0f2f1; border-radius: 5px; padding: 10px; margin-top: 10px; border-left: 3px solid #009688;">
                <p style="margin: 0; font-weight: 500;">
                    Extended hypothesis <span style="color: #00695c; font-weight: bold;">SUPPORTED</span> through chi-square testing and correlation analysis.
                </p>
            </div>
        </div>
    </div>
</div>

<div style="margin: 30px 0; background-color: #e0f2f1; border-radius: 8px; padding: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
    <h2 style="color: #00695c; border-bottom: 2px solid #80cbc4; padding-bottom: 8px;">Implications and Future Research</h2>
    
    <div style="background-color: white; border-radius: 8px; padding: 20px; margin-top: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
        <div style="display: flex; flex-wrap: wrap; gap: 20px;">
            <div style="flex: 1; min-width: 250px;">
                <h4 style="color: #00695c; border-bottom: 1px solid #b2dfdb; padding-bottom: 5px;">Policy Implications</h4>
                <ul style="padding-left: 20px; line-height: 1.6;">
                    <li>Differentiated climate responsibilities based on economic capacity</li>
                    <li>Technology transfer mechanisms to support lower-GDP countries</li>
                    <li>Focus on decoupling economic growth from emissions</li>
                </ul>
            </div>
            
            <div style="flex: 1; min-width: 250px;">
                <h4 style="color: #00695c; border-bottom: 1px solid #b2dfdb; padding-bottom: 5px;">Business Strategy</h4>
                <ul style="padding-left: 20px; line-height: 1.6;">
                    <li>Investment opportunities in clean technology</li>
                    <li>Carbon risk assessment frameworks</li>
                    <li>ESG integration aligned with economic transitions</li>
                </ul>
            </div>
            
            <div style="flex: 1; min-width: 250px;">
                <h4 style="color: #00695c; border-bottom: 1px solid #b2dfdb; padding-bottom: 5px;">Future Research</h4>
                <ul style="padding-left: 20px; line-height: 1.6;">
                    <li>Temporal analysis of progress toward net-zero targets</li>
                    <li>Sectoral breakdown of emissions by GDP category</li>
                    <li>Policy effectiveness in high vs. low GDP contexts</li>
                </ul>
            </div>
        </div>
    </div>
</div>

<div style="margin: 30px 0; background-color: #e0f2f1; border-radius: 8px; padding: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
    <h2 style="color: #00695c; border-bottom: 2px solid #80cbc4; padding-bottom: 8px;">Data Sources</h2>
    
    <div style="margin-top: 15px; line-height: 1.6;">
        <div style="background-color: white; padding: 15px; border-radius: 8px; margin-bottom: 15px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
            <h4 style="color: #00695c; margin-top: 0;">GDP per Capita:</h4>
            <ul style="margin-bottom: 0; padding-left: 20px;">
                <li><strong>Source:</strong> World Bank and OECD national accounts data (2025)</li>
                <li><strong>Dataset:</strong> gdp-per-capita-worldbank-constant-usd.csv</li>
                <li><strong>URL:</strong> <a href="https://ourworldindata.org/" style="color: #00897b; text-decoration: none; border-bottom: 1px dotted #00897b;">https://ourworldindata.org/</a></li>
            </ul>
        </div>
        
        <div style="background-color: white; padding: 15px; border-radius: 8px; margin-bottom: 15px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
            <h4 style="color: #00695c; margin-top: 0;">CO₂ Emissions per Capita:</h4>
            <ul style="margin-bottom: 0; padding-left: 20px;">
                <li><strong>Source:</strong> Global Carbon Budget (2024), Population based on various sources (2024) – with major processing by Our World in Data</li>
                <li><strong>Dataset:</strong> co-emissions-per-capita.csv</li>
                <li><strong>URL:</strong> <a href="https://ourworldindata.org/grapher/co-emissions-per-capita" style="color: #00897b; text-decoration: none; border-bottom: 1px dotted #00897b;">https://ourworldindata.org/grapher/co-emissions-per-capita</a></li>
            </ul>
        </div>
        
        <div style="background-color: white; padding: 15px; border-radius: 8px; margin-bottom: 15px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
            <h4 style="color: #00695c; margin-top: 0;">Net Zero Carbon Emissions Targets:</h4>
            <ul style="margin-bottom: 0; padding-left: 20px;">
                <li><strong>Source:</strong> Energy and Climate Intelligence Unit, Data-Driven EnviroLab, NewClimate Institute, Oxford Net Zero - Net Zero Tracker (2023) – with minor processing by Our World in Data</li>
                <li><strong>Dataset:</strong> Status of net-zero carbon emissions targets</li>
                <li><strong>URL:</strong> <a href="https://ourworldindata.org/" style="color: #00897b; text-decoration: none; border-bottom: 1px dotted #00897b;">https://ourworldindata.org/</a></li>
            </ul>
        </div>
    </div>
</div>

<div style="margin: 30px 0;">
    <h3 style="color: #00695c; border-bottom: 2px solid #80cbc4; padding-bottom: 8px;">Academic References</h3>
    <div style="background-color: #f5f5f5; padding: 15px; border-radius: 8px; font-style: italic; color: #555;">
        [Students will add citations for academic papers and research used in analysis]
    </div>
    
    <h3 style="color: #00695c; border-bottom: 2px solid #80cbc4; padding-bottom: 8px; margin-top: 20px;">Additional Sources</h3>
    <div style="background-color: #f5f5f5; padding: 15px; border-radius: 8px; font-style: italic; color: #555;">
        [Students will add citations for policy documents, reports, and other external research cited]
    </div>
</div>

<div style="background-color: #e0f2f1; border-radius: 8px; padding: 20px; margin-top: 30px; border-top: 5px solid #009688; text-align: center;">
    <p style="margin-bottom: 5px;"><strong>Assignment completed by:</strong> [Student Name]</p>
    <p style="margin-bottom: 5px;"><strong>Date:</strong> [Completion Date]</p>
    <p style="margin-bottom: 0;"><strong>Word Count:</strong> [Estimated word count for text sections]</p>
</div>